## Jupyter Notebook: Code for extracting and processing Zygo/MetroPro data files
### Simulating bonds between surface maps, auto-comparison of possible bonded pairs
##### Scott Gallacher (2315629), 2020/2021

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as Rot  # - performing rotations of vectors
import scipy.optimize
import scipy.ndimage  # - performing rotation of array as image - i.e. around z-axis
import copy  # - copy a python object
import matplotlib.pyplot as plt
import matplotlib.cm as cm  # - get unique colour maps
from itertools import combinations  # - get all N-length combinations from a list
from scipy.interpolate import griddata  # - possible interpolation function to fit new x,y positions to original x,y of grid
# %matplotlib notebook  # - to make plots interactive

In [2]:
# - 1x class: "ZygoMap(filename=None, array=None, map1=None, map2=None, angle=None)"
# - 2(+)x functions to work with pairs of maps:
# - - - "combinemaps(lowermap,uppermap,optimised=True,output=True)" - lowermap,uppermap are ZygoMap objects
# --------- combinemaps uses ztestf() so this is also a distinct function, while testf() (for flattening) can be a class method
# --------- could be defined within combinemaps (?) as it's only used there
# - - - "matchdims(map1,map2)" - map1,map2 are MxN arrays (used from within combinemaps(), using map.heights array)
# - - - (possible addition) - to simulate realistic contacts of surfaces when combined (e.g. find 3 points) (would be used by combinemaps())
# - - - "comparebonds" - to simulate interface (bond) of each combination of available maps, display lowest PV & RMS heights
# - - - "rotatepoints" - general rotation function made, but has no real use for ZygoMaps as each rotation method has unique needs

In [1]:
class ZygoMap:
    #structure: definition of methods for the class (to perform operations on the ZygoMap object - referred to as "self"):
    # - "zygoread(self, filename)" - extracting header fields and measurement data from MetroPro .txt files
    # ------------------------------ note that the data is not parsed into variables yet, just returned in more useful format
    # - "crop(self, radius=0)" - (for "heights" array) remove data outwith the radius (no crop by default) and centre on the valid data
    # - "testf(self, *args)" - test/loss function, passed to optimisation algorithm within rotateflat()
    # - "rotateflat(self, array=None)" - tilt removal by rotating until "flat" (minimising peak-to-valley height)
    #########
    # - "__str__(self)" - special method which tells Python how to display the object e.g. when calling print(ZygoMap)
    # ------------------- will print out a brief summary of the current object, depending on how it was created
    #########
    # - "__init__(self, filename=None, array=None, map1=None, map2=None, angle=None, flatten=True)"
    # -- special method which is always run first by Python when creating object
    # -- the main function of the class, initialises the object and performs the pre-processing steps using the methods listed above
    # -- any options given during e.g. "ZygoMap(option1=..., option2=...,)" get passed straight to __init__()
    
    def __init__(self, filename=None, array=None, map1=None, map2=None, angle=None, flatten=True):
        #initialise ZygoMap object, process to remove tilts and store information about surface and/or from file header
        #2 ways to make ZygoMap object:
        # - 1) reading in header and data arrays from ASCII .txt file (MetroPro formatting - see reference guide)
        # - -  defines a map for a single component, as specified by file (use "filename" keyword)
        # - 2) supplying an array directly to be processed (using "array" keyword)
        # - -  particularly needed to store the interface map for combinations of other maps and their optimal angle
        # - -  also allows simple user-defined MxN arrays to be provided
        
        #initialise all variables, whether None or otherwise
        #they can all be checked for None elsewhere, but it's best to make sure they exist
        self.filename = filename
        self.array = array
        self.map1 = map1
        self.map2 = map2
        self.angle = angle
        self.flatten = flatten
        
        if (filename is None) & (array is None):
            raise AttributeError("""Cannot construct map object - please explicitly supply either
                                 \"filename\" as a local file location or
                                 \"array\" as a 2D numpy array
                                 as arguments to begin processing.""")
            
        elif (filename is not None) & (array is not None):
            filename = None
            print("""Warning: Both \"filename\" and \"array\" arguments were provided. The initialisation will proceed using \"array\" argument.""")
        
        if array is not None:
            if isinstance(array, np.ndarray):
                if (len(array.shape) == 2) & (0 not in array.shape):
                    self.heights = array.copy()
                else:
                    raise ValueError("Invalid array dimensions - NxM 2D array shape required.")
            else:
                raise AttributeError("""Could not process given argument. \"array\" argument requires an NxM 2D numpy array.""")
                
        
        
        if filename is not None:
            self.filename = filename
            #get header and data from file by user-defined function "zygoread" (change to class method ?)
            fields, data = self.zygoread(filename)

            #header extraction
            self.stringConstant = fields[0][0]
            chunk = fields[0][1].split()
            self.softwareType, self.majorVersion, self.minorVersion, self.bugVers = [int(n) for n in chunk]

            self.softwareDate = fields[1][0]

            chunk = fields[2][0].split()
            self.intensOriginX, self.intensOriginY, self.intensWidth, self.intensHeight, self.Nbuckets, self.intensRange = [int(n) for n in chunk]

            chunk = fields[2][1].split()
            self.phaseOriginX, self.phaseOriginY, self.phaseWidth, self.phaseHeight = [int(n) for n in chunk]

            self.comment = fields[3][0]

            self.partSerNum = fields[4][0]
            self.partNum = fields[5][0]

            chunk = fields[6][0].split()
            self.source = int(chunk.pop(0))  # - want 1st and last separately (they are int, rest are float)
            self.timeStamp = int(chunk.pop(-1))  # - use .pop(index) to separate the item from the list
            self.intfScaleFactor, self.wavelengthIn, self.numericAperture, self.obliquityFactor, self.magnification, self.cameraRes = [float(n) for n in chunk]

            chunk = fields[6][1].split()
            self.cameraWidth, self.cameraHeight, self.systemType, self.systemBoard, self.systemSerial, self.instrumentId = [int(n) for n in chunk]

            self.objectiveName = fields[7][0]

            #want both index 6 & 7 seperately, as they need to be floats
            #convert the rest to int as before
            chunk = fields[8][0].split()  # - looks messier but should use this throughout to reduce repeated splitting
            self.targetRange = float(chunk.pop(6))  # - remove item at index 6 and returns it (and modifies original list)
            self.lightLevel = float(chunk.pop(6))  # - do it again as the index 7 is now at index 6 in the modified list
            self.acquireMode, self.intensAvgs, self.PZTCal, self.PZTGain, self.PZTGainTolerance, self.AGC, self.minMod, self.minModPts = [int(n) for n in chunk]

            chunk = fields[8][1].split()
            self.disconFilter = float(chunk.pop(4))
            self.phaseRes, self.phaseAvgs, self.minimumAreaSize, self.disconAction, self.connectionOrder, self.removeTiltBias, self.dataSign, self.codeVType = [int(n) for n in chunk]

            self.subtractSysErr = int(fields[8][2])

            self.sysErrFile = fields[9][0]

            chunk = fields[10][0].split()
            self.refractiveIndex, self.partThickness = [float(n) for n in chunk]

            self.zoomDesc = fields[11][0]


            #extract intensity and phase data as numpy arrays (reshape to header parameters)
            self.intensitymap = np.array(data[0], dtype=float).reshape(self.intensHeight, self.intensWidth)
            self.phasemap = np.array(data[1], dtype=float).reshape(self.phaseHeight, self.phaseWidth)

            #handle invalid values (given in MetroPro manual)
            self.intensitymap[self.intensitymap >= 64512] = np.nan
            self.phasemap[self.phasemap >= 2147483640] = np.nan

            #create arrays in terms of number of waves, and height itself (in metres)
            #by given formula
            if self.phaseRes == 0:
                self.R = 4096
            elif self.phaseRes == 1:
                self.R = 32768
                
            #conversion formula from MetroPro manual
            self.waves = self.phasemap*(self.intfScaleFactor*self.obliquityFactor)/self.R
            self.heights = self.waves*self.wavelengthIn
             
        elif array is not None:
            #allow map object to be created from scratch (i.e. make interface as a map object directly)
            
            #set info about interface's source maps and their combination
            #leave default as None, this is only applicable to interfaces created out of combinemaps
            #which provides the 2 filenames and optimised angle
            #test if this is a combination of maps (interface) or user-defined single map ("map1","map2","angle" do not apply)
            if None not in (map1,map2,angle):
                self.map1 = map1
                self.map2 = map2
                self.angle = angle
            self.heights = array.copy()
            
        
        ########################
        ########################
        ##pre-processing maps
        
        #grid points for use in some methods (?) (just using array i,j position index (can scale later))
        self.y, self.x = np.indices(self.heights.shape)
        
        #apply cropping first (user-defined radius ? or default ?)
        #self.cropped = self.crop(self.heights)
        #orderings/logistics of this needs fixed: which array is edited? when? what effect should user cropping give?
        #create a copy of the initial height array, this allows crop to act on those values and provide new self.heights
        #without data loss
        self.heights0 = self.heights.copy()
#         self.cropped = self.heights[:]  # - slice notation actually still links the variables, need np.copy() instead
        #store initial attributes just so they are not missing at any point
        #they will be inaccurate initially (e.g. based on tilted map), but get updated via rotateflat()
        self.peak, self.valley = np.nanmax(self.heights0), np.nanmin(self.heights0)
        self.peakvalley = self.peak - self.valley
        self.rms = np.sqrt(np.nanmean(self.heights0**2))
        
        #adjust to centre of valid points (centre of surface)
        self.validrows, self.validcols = np.where(np.isfinite(self.heights0))
        self.centre = int(np.nanmean(self.validrows)), int(np.nanmean(self.validcols))
        self.x -= self.centre[1]
        self.y -= self.centre[0]
        
                
        #remove tilt if present
        #note: added "flatten" keyword to allow user to use the map as read from file
        #still cropped to centre the view, but without removing any tilt
        #and also make sure to adjust valid rows & columns for this fully flattened array
        #use a 3rd heights array - a flattened but not cropped version
        #so will have: self.heights0 - the original data, self.heights1 - flattened version, self.heights - flattened and cropped to centre on valid area
        self.heights1 = self.heights0.copy()
        if flatten:
            self.heights1 = self.rotateflat(self.heights1)
            self.validrows, self.validcols = np.where(np.isfinite(self.heights1))  # - update the valid points for flattened map
            
            
            self.heights = self.heights1.copy()  # - keep heights1 stored; use heights as main array object for further uses
        
        self.crop()
        
        return
    
    #use __str__ method to provide user summary on calling print(ZygoMap)
    #three possible paths, depending if single file object; interface created of two maps; or simply a user-defined array
    def __str__(self):
        if hasattr(self, "filename"):
            return "ZygoMap object for file: {0}. \nPeak-to-valley height: {1:.1f} nm \nRMS height: {2:.1f} nm".format(repr(self.filename), self.peakvalley*1e9, self.rms*1e9)
        elif all(hasattr(self, attr) for attr in ("map1","map2","angle")):
            return "ZygoMap interface object for {0} & {1} combined at angle {2:.0f} degrees. \nPeak-to-valley height of bond: {3:.4e} m \nRMS height of bond: {4:.4e} m".format(repr(self.map1), repr(self.map2), self.angle, self.peakvalley, self.rms)
        else:
            return "ZygoMap object for user-provided array. \nPeak-to-valley height: {0:.4e} m \nRMS height: {1:.4e} m".format(self.peakvalley,self.rms)
        
#     @staticmethod
    def zygoread(self, filename):
        #works with specific ASCII format .txt files (documented in MetroPro reference guide)
        with open(filename, "r") as f:
            fstrings = f.read().split("\"")  # - split by qoutation marks (easier to seperate string fields from data)

            fields = []
            data = []
            section = 0
            for elt in fstrings:
                if "#" in elt:  # - use this test to show end of header section, then switch to next section
                    section += 1
                    pass

                if section == 0:  # - processing header section
                    #multiple fields are stored within single strings, so need to split by newline to narrow down
                    #numbers stored within strings can be extracted afterwards
                    #excess artifacts can be filtered out using if (True) test on elements of split string
                    #fails on empty string, thus keeping only the relevant fields
                    elt = elt.split("\n")
                    values = [_ for _ in elt if _]  # - filter bad elements (e.g. "" which have no data and return False)
                    #test for non-empty lists (indicates no data was found in values list)
                    if values:
                        fields.append(values)

                elif section == 1:  # - move to data extraction for intensities and phases
                    #the initial split left the both datasets in a single string - separate by "#"
                    #split string containing a dataset then iterate through the resulting lines of 10
                    #append all values to a 'data' array, filter as before
                    #splitting by "#" will result in 2 lists stored in the overall 'data' list
                    #i.e. can extract: intensities = data[0], phases = data[1]
                    for line in elt.split("#"):
                        values = [_ for _ in line.split() if _]
                        if values:
                            data.append(values)
                        
        return fields, data
    
    def crop(self, radius=0):
        #allow user to crop to extract only data within some radius
        #most needed to avoid large edge effects (discontinuities)
        #use the (stored) centred x and y positions to check against radius
        #make a new cropped array where points outwith radius are set to nan
        #and "zoom in" to store only the array rows & columns within the valid range
        #will run during __init__(), with default radius = 0, so can avoid editing if radius is default
        #and only do if user chose a (non-zero) radius
        #thus only the centring of view by array slicing is performed (no need for separate functions)
        
        #set cropped array based on original state of heights (so not cropping multiple times and losing data)
#         cropped = self.heights0.copy()
        cropped = self.heights1.copy()
        
        if radius != 0:
            #if radius non-zero, we will be setting valid points to invalid (nan)
            #use centred x,y grid points for full data array to make a mask for points outside radius
            #then just change these points to nan (thus matching the pre-existing background)
            outsideR = self.x**2 + self.y**2 > radius**2
            cropped[outsideR] = np.nan
            
            #could add a flag/callback here to automatically re-apply rotations after crops (otherwise advise user to do it)
            #e.g. if callback = True -> rotateflat()
        
        #now find the extreme bounds of valid points and slice the array to show only the data within
        validrows, validcols = np.where(np.isfinite(cropped))
        lft, rgt, upp, low = np.nanmin(validcols), np.nanmax(validcols), np.nanmin(validrows), np.nanmax(validrows)
        
        
        cropped = cropped[upp:low+1, lft:rgt+1]
        self.heights = cropped.copy()
        
        #update valid positions, after rows/columns removed
        self.validrows, self.validcols = np.where(np.isfinite(self.heights))
        self.centre = int(np.nanmean(self.validcols)),int(np.nanmean(self.validrows))
        
        return cropped
    
    def testf(self, *args):
        #test function to be minimised by optimisation algorithm
        #uses rotations around x & y axes to minimise peak-to-valley height of map
        angx, angy = args[0]  # - format of scipy minimize requires a single 1st argument to alter - can have list with multiple
        array = args[1]  # - the simple 2D array of height values

        dims = array.shape
        x,y = np.indices(dims)  # - use row,column positions (i.e. matrix i,j) as the x,y for the position vectors
        
        #make array of position vectors in NxMx3 format: i.e. [x,y,height] as a single element for one point in 3D
        #use dstack to arrange the separate array values down each column, then reshape to (NxM)x3 array
        x -= self.centre[0]  # - to ensure rotations around centre (add back after rotating)
        y -= self.centre[1]
        centreOffset = array[self.centre[0],self.centre[1]]  # - remove offset of centre height (makes centre the origin)
        array -= centreOffset
        
        vectarray = np.dstack([x,y,array])
        vectarray = vectarray.reshape(vectarray.size//3, 3)

        #define rotations around x and y axes respectively and combine via * operation
        rx = Rot.from_rotvec(angx*np.array([1,0,0]))
        ry = Rot.from_rotvec(angy*np.array([0,1,0]))
        r = rx*ry

        #apply overall rotation to entire vector array (rotates each vector individually)
        #using rounding of the new x,y (the row,column values) to approximate positions to grid
        #NOTE: actually not necessary to recreate grid for testf, no rounding needed
        #only have to calculate peakvalley here, and can do that just from the new height values (newarray[:,2])
        newarray = r.apply(vectarray)
#         newarray[:,0] = np.round(newarray[:,0])
#         newarray[:,1] = np.round(newarray[:,1])
        #retain only the x,y inside the original grid shape (otherwise have to extend to arbitrary rows/columns)
#         newarray = newarray[(newarray[:,0] < dims[0]) & (newarray[:,1] < dims[1])]
        
        #create "empty" array of nan, matching original grid
        #then set values directly by using integer conversion of x,y columns
#         newheights = np.ones(dims) * np.nan
#         newheights[newarray[:,0].astype("int"), newarray[:,1].astype("int")] = newarray[:,2]
#         newheights = newarray[:,2].reshape(dims)  # - old, inaccurate method

        #check the peak to valley height of rotated array
        #note: may use a best-fit plane to the array (e.g. reducing sum of squared deviations across entire array)
        #would potentially not need a test function for this (?) as the minimisation would be done by fitting algorithm
        peakvalley = np.nanmax(newarray[:,2]) - np.nanmin(newarray[:,2])

        return peakvalley
    
    def rotateflat(self, array=None):
        #apply minimization of peak-to-valley height for rotations around x & y axes
        #call to external testf() (or make testf internal ?)
        #then apply best rotation and return rotated array
        #"array" argument left so normal or cropped maps can be used (i.e. self.heights vs self.cropped)
        #detect array=None to mean default self.heights
        if array is None:
            array = self.heights
        
        #minimise peak to valley height for rotation angles around x-axis and y-axis
        #x0 gives the initial "guess" for the optimiser to use as x & y angles within testf()
        opts = {"ftol":1e-15, "xtol":1e-15, "maxiter":1000}
        params = scipy.optimize.minimize(self.testf, x0=[0,0], args=array, tol=1e-15, method="Nelder-Mead", options=opts)
        angx,angy = params["x"]  # - access the optimal angles found by the minimisation
        
        
        #now apply the optimal rotations (same method as contained in test function, "testf")
        dims = array.shape
        x,y = np.indices(dims)
        x -= self.centre[0]  # - to ensure rotations around centre (add back after rotating)
        y -= self.centre[1]
        centreOffset = 0  # - remove offset of centre height (makes centre the origin)
#         if np.isfinite(array[self.centre[0],self.centre[1]]):
#             centreOffset = array[self.centre[0],self.centre[1]]
#             print(1, centreOffset)
        
        #create array stack of vectors (MxNx3)
        vectarray = np.dstack([x,y,array])
        vectarray = vectarray.reshape(int(vectarray.size/3), 3)  # - reshape to (MxN)x3
        
        #define rotations along x & y axes with the specified angles, combine with * operation
        rotx = Rot.from_rotvec(angx*np.array([1,0,0]))
        roty = Rot.from_rotvec(angy*np.array([0,1,0]))
        r = rotx*roty
        
        newarray = r.apply(vectarray)
        #newheights = newarray[:,2].reshape(dims)  # - old, inaccurate method
        #approximate new x,y positions by rounding to the original grid (integers)
        newarray[:,0] = np.round(newarray[:,0]) + self.centre[0]
        newarray[:,1] = np.round(newarray[:,1]) + self.centre[1]
        newarray[:,2] += centreOffset
        #extract only the x,y inside the original grid shape (otherwise have to extend grid to handle arbitrary x,y values)
        #but should use the full array of rotated vectors to give accurate calculations
        newvalid = newarray[(newarray[:,0] >= 0) & (newarray[:,0] < dims[0]) & (newarray[:,1] >= 0) & (newarray[:,1] < dims[1])]
        
        #create "empty" nan array, matching original dimensions
        #then directly set values using integer conversion of x,y columns
        newheights = np.ones(dims) * np.nan
        newheights[newvalid[:,0].astype("int"), newvalid[:,1].astype("int")] = newvalid[:,2]
        
        #centre in z-axis
        newheights -= np.nanmean(newheights)
                
        #generate attributes for flattened map (should probably set these before this point)
        #i.e. in __init__() for initial array (in case of problems they would then still be defined in some way)
        self.peak, self.valley = np.nanmax(newarray[:,2]), np.nanmin(newarray[:,2]) # - need to consider all rotated values, even if they are not going to be stored (outwith bounds)
        self.peakvalley = self.peak - self.valley
        self.rms = np.sqrt(np.nanmean(newarray[:,2]**2))
        
        #update the stored heights array
        self.heights = newheights.copy()
        
        return newheights
    

In [4]:
def ztestf(*args):  # - optimisation of rotation around z-axis of upper map w.r.t lower map
    angz = args[0][0]  # - optimize.minimize gives angz as [0.] (why?) so needs extracted from list as well as args tuple
    lowermap, uppermap = args[1],args[2]

    dims = uppermap.shape
    
    #now get new rotated array (rotating around z/in x-y plane)
    #using scipy.ndimage.rotate()
    #order=0 means no additional interpolation of values when rotating
    #reshape=False maintains original dimensions (used in case of rotating outside of original shape - not applicable here)
    #use mode="constant" and cval=np.nan to fill out all (and perhaps new) invalid points with nan
    newheights = scipy.ndimage.rotate(uppermap, angz, order=0, reshape=False, mode="constant", cval=np.nan)
    
    #simulate the surface contact, using simple 1 point of contact
    #one map flipped horizontally and negated, their addition describes the interface heights
    interface = -newheights[::-1] + lowermap
    interface += abs(np.nanmin(interface))  # - set contact point to be zero height (negative values are non-physical intersection of surfaces)
    interface -= np.nanmean(interface)
    
    #again minimising peak-to-valley height, though a sum of squares approach could be used
    peakvalley = np.nanmax(interface) - np.nanmin(interface)
    
    return peakvalley

In [5]:
def matchdims(map1,map2):
    #given two arrays (not map objects), truncate them to their lowest shared dimensions to be able to sum them
    #note: should probably choose to add rows/columns rather than remove data
    m1,m2 = map1.copy(),map2.copy()
    
    m1dims, m2dims = m1.shape, m2.shape
    diffs = [(m1dims[0] - m2dims[0]), (m1dims[1] - m2dims[1])]  # - find difference in number of rows & columns between arrays
    
    #two types of slices for which dimension is the largest between the two maps
    #use modulo % to check divisibility, // to do whole number division
    #then slice from each end of array (avoid bias/truncating on one side)
    #take the divisor result from both sides, then take the remainder from end of array
    #balances as much as possible, but odd-numbered differences will be asymmetric (just take remaining 1 from end of array)
    #use (len(array) - number) to do backwards slice - needed for minus zero slice which is treated as zero
    #using len() gives an absolute index rather than relative
    #addition of abs() makes things easier
    
    #for rows
    rem,div = (diffs[0] % 2), (diffs[0]//2)
    if m1dims[0] > m2dims[0]:
        m1 = m1[abs(div):m1.shape[0]-abs(div+rem),:]
    elif m1dims[0] < m2dims[0]:
        m2 = m2[abs(div):m2.shape[0]-abs(div+rem),:]
        
    #for columns
    rem,div = (diffs[1] % 2), (diffs[1]//2)
    if m1dims[1] > m2dims[1]:
        m1 = m1[:, abs(div):m1.shape[1]-abs(div+rem)]
    elif m1dims[1] < m2dims[1]:
        m2 = m2[:, abs(div):m2.shape[1]-abs(div+rem)]
    
    return m1,m2

In [6]:
def combinemaps(lowermap, uppermap, optimised=True, output=True):
    m1,m2 = lowermap.heights, uppermap.heights
    #function to combine ZygoMap objects
    #flips and negates values of the 2nd map "uppermap"
    #emulating the surface placed faced down on the other
    #returns ZygoMap object from the combination of the lower map and the transformed uppermap
    #where the magnitude of largest negative has been added back as an offset to prevent non-physical overlap of surfaces
    
    #check for equal shapes (changed to automatically crop to smallest shared values)
    #use matchdims() function, will return arrays with equal rows,columns for direct addition of arrays
    if m1.shape != m2.shape:
        m1,m2 = matchdims(m1,m2)
    
    #find optimised angle of rotation (of uppermap with respect to lowermap)
    #minimise the peakvalley height with rotation angle around z-axis
    #NOTE: previous rotation method not working the same for z rotations
    #using scipy.ndimage.rotate (with order=0 to maintain array values (no spline interpolation))
    #large angular range needed -> need brute() function to get the accurate value
    #define angular range by ranges=(slice(0,360),) (the slice object is preferred by brute function definition)
    optimalangle = 0
    if optimised == True:
        optimalangle = scipy.optimize.brute(ztestf, ranges=((slice(0,360),)), args=(m1,m2))[0]
    
        #apply this angle with scipy.ndimage.rotate() to get new array of the rotated uppermap
        #use this array directly for the interface (in place of uppermap.heights), and leave each individual map untouched
        #should store some indicator for user of the optimal angle used (property of interfacemap ?)
        newheights = scipy.ndimage.rotate(m2, optimalangle, order=0, reshape=False, mode="constant", cval=np.nan)
    else:
        newheights = m2.copy()

    #flip and negate uppermap, then add to lower map for interface
    #note: using only simple one-point contact
    #for more realistic simulation, want to find 3 points or simulate how the upper surface would "settle" onto lower
    #experimented (manually simulating 2 rotations based on position of point around centre)
    #but not a clear successful method, would leave as future work
    interface = -newheights[::-1] + m1
    interface += abs(np.nanmin(interface))  # - add back largest overlap, to leave maps just touching
    interface -= np.nanmean(interface)  # - centre in z-axis around mean
        
    #construct as ZygoMap object
    #providing basic details about its construction (the combination of which maps at what angle)
    if hasattr(lowermap, "filename") and hasattr(uppermap, "filename"):
        interfacemap = ZygoMap(array=interface, map1=lowermap.filename, map2=uppermap.filename, angle=optimalangle)
    else:
        interfacemap = ZygoMap(array=interface)
    
    #give user some knowledge on the optimised set-up
    #this could be moved elsewhere possibly for better access ->  added __str__() method to allow user to print() attributes
    #added flags for optimisation and output
    #set output to false for auto-comparison, so can display at end in a sorted order
    if optimised and output:
        if hasattr(lowermap, "filename") and hasattr(uppermap, "filename"):  # - if each defined from files, can use their filenames as references
            print("Maps combined for optimal angle of {0:.2f} degrees\n\
            {1} clockwise w.r.t {2}".format(optimalangle,uppermap.filename,lowermap.filename))
        else:
            print("Maps combined for optimal angle of {0:.2f} degrees\n\
            (map2 clockwise w.r.t map1)".format(optimalangle))
        print("Optimised peak-to-valley height: {0} m".format(interfacemap.peakvalley))
        print("Optimised RMS height: {0} m".format(interfacemap.rms))
        
    
    return interfacemap

In [13]:
def comparebonds(zmaps, sort="both", plot=False):
    #NOTE: using itertools.combinations module
    #for a list (or maybe dict as well ?) of map objects, iterate through every pair combination
    #comparing bonds by peak-to-valley height & RMS height
    #additionally, can print information about optimal angle (around z-axis) to combine each pair
    
    #attain array from generator object returned by combinations (using n=2 items per combo)
    mpairs = np.array(list(combinations(zmaps, 2)))
    
    #combine maps, creating interface object for each pair (of ZygoMap class, defined with array rather than filename)
    #will store the PV & RMS values for each in numpy arrays, to then be sorted best to worst
    #choice to store the interfaces ? - for low number of arrays this should be ok
    
    #initialise empty arrays which will store the PV/RMS as calculated (necessary if not storing each interface)
    pvValues = np.zeros(len(mpairs))
    rmsValues = np.zeros(len(mpairs))
    
    interfacemaps = np.zeros(len(mpairs), dtype=object)  # - to store each combo pair's interfacemap (ZygoMap object)
    #note: may not be so simple - only local to function, may need to either return interfacemaps or create as global variable
    for i in range(len(mpairs)):
        combo = mpairs[i]
        interfacemap = combinemaps(*combo, output=False)
        pvValues[i] = interfacemap.peakvalley
        rmsValues[i] = interfacemap.rms
        
        interfacemaps[i] = interfacemap
        
    #now, sort for display
    #sort the mpairs list differently for either pv or rms, storing each separately
    #allow "sort" keyword to limit comparison to only pv or only rms (defaults to "both")
    
    #sort combos by lowest peak-valley height
    if sort in ("both","pv"):
        lowestpv = mpairs[pvValues.argsort()]
        print("Sample Bonds sorted by lowest peak-to-valley height:\n")
        for i in range(len(lowestpv)):
            combo = lowestpv[i]
            print("{} - ".format(i+1),*[m.filename for m in combo])
            print("Peak-to-valley Height: {0:.1f} nm".format(pvValues[pvValues.argsort()][i] * 1e9))  # - convert to nanometres
            print("RMS Height: {0:.1f} nm".format(rmsValues[pvValues.argsort()][i] * 1e9))
            print("\n")
    
    #sort combos by lowest rms height
    if sort in ("both","rms"):
        lowestrms = mpairs[rmsValues.argsort()]
        print("Sample Bonds sorted by lowest RMS height:\n")
        for i in range(len(lowestrms)):
            combo = lowestrms[i]
            print("{} - ".format(i+1),*[m.filename for m in combo])
            print("RMS Height: {0:.1f} nm".format(rmsValues[rmsValues.argsort()][i] * 1e9))
            print("Peak-to-valley Height: {0:.1f} nm".format(pvValues[rmsValues.argsort()][i] *1e9))
            print("\n")
            
    if plot is True:
        if sort in ("both","pv"):
            #sort the found pv & rms heights, ordering by the lowest pv in both cases (keep both values aligned per combo)
            plt.figure(figsize=(10,6))
            plt.plot(pvValues[pvValues.argsort()], "o", label="Peak-Valley")
            plt.plot(rmsValues[pvValues.argsort()], "o", label="RMS")
            plt.title("Bond height values comparison (sorted by lowest peak-valley value)\n")
            
            plt.xlabel("Maps used in simulated bond")
            plt.ylabel("Height of bond interface (nm)")
            plt.xticks(range(len(mpairs)), np.array([m.map1.split(".")[0]+"\n"+m.map2.split(".")[0] for m in interfacemaps])[pvValues.argsort()], rotation=0)
            plt.yticks(np.arange(0, max(pvValues) + 10e-9, 10e-9))
            plt.ticklabel_format(axis="y", style="sci", scilimits=(-9,-9), useMathText=True)
            plt.axhline(60e-9, linestyle="solid", linewidth=3, color="r")
            plt.legend(loc="upper left", fontsize=14)
            plt.grid()
            plt.show()
            
        if sort in ("both","rms"):
            #sort the found pv & rms heights, ordering by the lowest rms in both cases (keep both values aligned per combo)
            plt.figure(figsize=(10,6))
            plt.plot(pvValues[rmsValues.argsort()], "o", label="Peak-Valley")
            plt.plot(rmsValues[rmsValues.argsort()], "o", label="RMS")
            plt.title("Bond height values comparison (sorted by lowest RMS value)\n")
            
            plt.xlabel("Maps used in simulated bond")
            plt.ylabel("Height of bond interface (nm)")
            plt.xticks(range(len(mpairs)), np.array([m.map1.split(".")[0]+"\n"+m.map2.split(".")[0] for m in interfacemaps])[rmsValues.argsort()], rotation=0)
            plt.yticks(np.arange(0, max(pvValues) + 10e-9, 10e-9))
            plt.ticklabel_format(axis="y", style="sci", scilimits=(-9,-9), useMathText=True)
            plt.axhline(60e-9, linestyle="solid", linewidth=3, color="r")
            plt.legend(loc="upper left", fontsize=14)
            plt.grid()
            plt.show()
            
    return  # - nothing right now (simplest for usability) but could offer the sorted arrays and all interfacemaps